In [19]:
import os
import torch
import mediapipe as mp
import numpy as np
import cv2
from protobuf_to_dict import protobuf_to_dict

In [20]:
def make_Video_and_tensor(input_path, output_video_path, output_tensor_path):

    # Prepare DrawingSpec
    mp_drawing = mp.solutions.drawing_utils 
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

    # Config holistic
    mp_holistic = mp.solutions.holistic
    holistic = mp_holistic.Holistic(
        min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
    # 영상 가져오기
    cap = cv2.VideoCapture(input_path)

    # 영상 저장
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') # 영상 포맷
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (1280,720)) # 비디오 경로, 영상 포맷, 초당 프레임, width*height 

    # 영상 width, height 설정
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    # 각 요소(왼손, 오른손, 얼굴, 포즈) 좌표 저장 리스트
    left_hand_lists = []
    right_hand_lists = []
    face_lists = []
    pose_lists = []

    while cap.isOpened():

        success, image = cap.read() 

        if not success: # 동영상 끝
            break

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = holistic.process(image)

        # Draw landmark annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
          image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
        mp_drawing.draw_landmarks(
          image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(
          image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(
          image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        # 왼손 랜드마크 리스트에 저장
        if results.left_hand_landmarks: # 영상에 왼손이 잡힐 경우
            left_hand_list = []
            for lm in results.left_hand_landmarks.landmark:
                left_hand_list.append([lm.x, lm.y, lm.z])
            left_hand_lists.append(left_hand_list)
        else: # 영상에 오른손이 잡히지 않을 경우
            left_hand_lists.append([[0] * 3 for _ in range(21)])
              
        # 오른손 랜드마크 리스트에 저장
        if results.right_hand_landmarks: # 영상에 오른손이 잡힐 경우
            right_hand_list = []
            for lm in results.right_hand_landmarks.landmark:
                right_hand_list.append([lm.x, lm.y, lm.z])
            right_hand_lists.append(right_hand_list)
        else: # 영상에 오른손이 잡히지 않을 경우
            right_hand_lists.append([[0] * 3 for _ in range(21)])

        # 얼굴 랜드마크 리스트에 저장
        if results.face_landmarks: # 영상에 얼굴이 잡힐 경우
            face_list = []
            for lm in results.face_landmarks.landmark:
                face_list.append([lm.x, lm.y, lm.z])
            face_lists.append(face_list)
        else: # 영상에 얼굴이 잡히지 않을 경우
            face_lists.append([[0] * 3 for _ in range(468)])

        # 포즈 랜드마크 리스트에 저장
        if results.pose_landmarks: # 영상에 포즈가 잡힐 경우
            pose_list = []
            for lm in results.pose_landmarks.landmark:
                pose_list.append([lm.x, lm.y, lm.z])
            pose_lists.append(pose_list)
        else: # 영상에 포즈가 잡히지 않을 경우
            pose_lists.append([[0] * 3 for _ in range(33)])
            

        out.write(image)  # 실질적인 영상 쓰기   

    # 텐서로 변환
    lt = torch.FloatTensor(left_hand_lists)
    rt = torch.FloatTensor(right_hand_lists)
    ft = torch.FloatTensor(face_lists)
    pt = torch.FloatTensor(pose_lists)
    
    # 텐서 저장할 디렉토리 생성
    if not(os.path.isdir(output_tensor_path)):
        os.mkdir(output_tensor_path)
        
    # 텐서 파일 저장
    torch.save(lt, output_tensor_path+"lt.pt")
    torch.save(rt, output_tensor_path+"rt.pt")
    torch.save(ft, output_tensor_path+"ft.pt")
    torch.save(pt, output_tensor_path+"pt.pt")
    
    holistic.close()
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [21]:
# 디렉토리 생성
if not(os.path.isdir("./output/video")):
    os.mkdir("./output/video")
if not(os.path.isdir("./output/tensor")):
    os.mkdir("./output/tensor")

for SignVideo in os.listdir("./video"):
    make_Video_and_tensor("./video/"+SignVideo, "./output/video/"+SignVideo, "./output/tensor/"+SignVideo[:-4]+"/")

In [14]:
print(torch.load("./output/tensor/KETI_SL_0000000002/lt.pt").shape)

torch.Size([160, 21, 3])


In [8]:
print(torch.load("./output/tensor/KETI_SL_0000000002/rt.pt").shape)

torch.Size([160, 21, 3])


In [9]:
print(torch.load("./output/tensor/KETI_SL_0000000002/pt.pt").shape)

torch.Size([160, 33, 3])


In [10]:
print(torch.load("./output/tensor/KETI_SL_0000000002/ft.pt").shape)

torch.Size([160, 468, 3])
